We can access BigQuery's public datasets directly. Simply describe the dataset and table name properly, and bingo! you can query the data.
You can browse through Bigquery's public datasets here: https://bigquery.cloud.google.com/publicdatasets/

### Section 1

Look at the tables - structure, sample data, run some queries on them


### Section 2

Query for specific types of images, then:
- get the original URL for each file for that type
- access, download and save the files
- copy the files to a specific cloud storage bucket

## Section 1
### Let's take a peek at the Open Images Dataset and run some queries on the tables

The Open Image dataset has four tables. For each of the 4, we will:

- list the # of rows
- peek into each one
- list the fields
- preview a sample of the data content
- on some of them, run other queries to validate the speed of BigQuery!

In [39]:
#import tools
import google.datalab.bigquery as bq
import matplotlib.pyplot as plot
import numpy as np
import pandas as pd
from pandas.io import gbq
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF

In [40]:
%%bq tables describe --name "bigquery-public-data.open_images.annotations_bbox"
#describe dataset.table fields, types, and if available -> mode, description

## Table 1: Annotations ("annotations_bbox")
Describes the attributes of the file - source, label name, confidence (of the label with the image), x and y mins and max, and other image parameters.

In [42]:
%%bq tables describe --name "bigquery-public-data.open_images.annotations_bbox"

In [43]:
bq.Query.from_table(table1).execute(sampling=bq.Sampling.default(
    fields=['image_id',
            'label_name',
            'confidence',
            'x_min',
            'y_min',
            'is_depiction'])).result()

image_id,label_name,confidence,x_min,y_min,is_depiction
0001a282f0638ef1,/m/0hnyx,1.0,0.122,0.001,
000323390354c939,/m/0284d,1.0,0.141,0.244,
0001a1b43b815ec2,/m/04dr76w,1.0,0.593,0.148,
00031197fb7b015d,/m/0c_jw,1.0,0.457,0.657,
00037418480b82d5,/m/081qc,1.0,0.457,0.686,


## Table 2: Dictionary ("dict")
Describes the label name, and the descriptive name, which is the one that makes sense to humans to describe the image!

In [44]:
table2=bq.Table('bigquery-public-data.open_images.dict')
table2.metadata.rows

19868

In [45]:
%%bq tables describe --name "bigquery-public-data.open_images.dict"

In [46]:
bq.Query.from_table(table2).execute(sampling=bq.Sampling.default(
    fields=['label_name',
            'label_display_name'])).result()

label_name,label_display_name
/m/01www,CD
/m/0h989,Go
/m/03bx7vb,Ox
/m/01lgkm,RV
/m/012wxt,Vj


In [47]:
%%bq query --name image_types
SELECT DISTINCT label_display_name as lname, COUNT(label_display_name)as namecount
FROM `bigquery-public-data.open_images.dict`
GROUP BY lname

In [48]:
image_types.execute().result()

lname,namecount
Ferrari 288 gto,1
Salt water taffy,1
Toboggan,1
800 metres,1
Geraniales,1
Sea captain,1
Ford crown victoria,1
Reciprocating saws,1
Forehead,1
Hyundai genesis coupe,1


## Table 3: Images ("images")
Describes the image URL, subset (eg., training), license, size, title and other features to access, retrieve, and use the actual file with which lable is associated.

In [49]:
table3=bq.Table('bigquery-public-data.open_images.images')
table3.metadata.rows

9178275

In [50]:
%%bq tables describe --name "bigquery-public-data.open_images.images"

In [51]:
bq.Query.from_table(table3).execute(sampling=bq.Sampling.default(
    fields=['subset',
            'original_url',
            'original_landing_url',
            'license',
            'title'])).result()

subset,original_url,original_landing_url,license,title
train,https://farm4.staticflickr.com/7162/6707198107_1993d73034_o.jpg,https://www.flickr.com/photos/spreadshirt/6707198107,https://creativecommons.org/licenses/by/2.0/,Men’s Pique Polo (NA)
train,https://c8.staticflickr.com/4/3676/11012143656_40a5269d42_o.jpg,https://www.flickr.com/photos/pawel_pacholec/11012143656,https://creativecommons.org/licenses/by/2.0/,Ladybug on green leaf
train,https://c4.staticflickr.com/9/8157/7599640374_f278fa5bd7_o.jpg,https://www.flickr.com/photos/biodivlibrary/7599640374,https://creativecommons.org/licenses/by/2.0/,n53_w1150
train,https://c7.staticflickr.com/5/4101/4861005456_71b3b0be00_o.jpg,https://www.flickr.com/photos/aliestelle/4861005456,https://creativecommons.org/licenses/by/2.0/,DSC_4918
train,https://farm5.staticflickr.com/5443/17832643436_904497178e_o.jpg,https://www.flickr.com/photos/usarmyafrica/17832643436,https://creativecommons.org/licenses/by/2.0/,Central Accord 2015


## Table 4: labels ("labels")
Lists the image id, source (whether it was labeled by humans or machine), label name, and the confidence of the label. Some of the fields are duplicated in other tables, such as image ids.

In [52]:
table4=bq.Table('bigquery-public-data.open_images.labels')
table4.metadata.rows

92665550

In [53]:
%%bq tables describe --name "bigquery-public-data.open_images.labels"

In [54]:
df2 = bq.Query.from_table(table4).execute(sampling=bq.Sampling.default(
    fields=['image_id',
            'source',
            'label_name',
            'confidence'])).result()
df2[0]

{u'confidence': 0.0,
 u'image_id': '001dd90fc5ae92c9',
 u'label_name': '/m/0dj6p',
 u'source': 'human'}

In [55]:
df2

image_id,source,label_name,confidence
001dd90fc5ae92c9,human,/m/0dj6p,0.0
0021653528cec6d7,human,/m/058hry,0.0
005e9907ccccfcf7,human,/m/0162v0,0.0
0079eddf36c14e63,human,/m/01hj56,0.0
008e0dc6b90fd509,human,/m/037p63,0.0


In [56]:
%%bq query --name sources
SELECT source AS sname, COUNT(source) as watchnum
FROM `bigquery-public-data.open_images.labels`
GROUP BY sname

In [57]:
sources.execute().result()

sname,watchnum
human,11629927
machine,81035623


## Section 2
### Get specific type of images to build Image repository for Processing
- Create a query specifying images labelled as "traffic signs"
- Get the urls for each of those files
- Access, download and save each of them
- Copy them to a storage bucket, presumably for Image processing or to build an AI model

In [58]:
#insert your project id here
#then create a query specifying what kind of images you want to see; I wanted traffic signs, so my query is pulling titles with traffic sign in their names
# then put the results in a dataframe (df1)
projectid = "hs-backup-updraft"
test_query = """SELECT original_url FROM [bigquery-public-data.open_images.images] WHERE title LIKE '%traffic sign%' GROUP BY original_url"""
df1 = gbq.read_gbq(test_query, projectid)

Requesting query... ok.
Job ID: afe7625e-2a1f-4efb-b093-0cfd1cdf9d2a
Query running...
Query done.
Cache hit.

Retrieving results...
Got 75 rows.

Total time taken 0.82 s.
Finished at 2018-10-16 20:51:39.


In [59]:
import urllib
import urllib2
bucket = 'gs://' + 'open-images-urls/' + '*.jpg'

In [63]:
bucket_storage = 'gs://' + 'open-images-urls/'

In [61]:
#take dataframe and make it a list to make for easier reading
url_list=df1['original_url'].tolist()
#convert from unicode to list of bytes,string, otherwise there is a 'u' in front of each list element
my_list = [str(url_list[x]) for x in range(len(url_list))]

#for each element of the list (each url containing 1 image of the type specified in the query)
#read the file
#initialize the counter y, then for each file save, update it, which also is used for adding to the file name that it will be saved as - for example, temp1, temp2 and so on
#once file is accessed through the url (website) and opened, save it for yourself to do ML image processing or whatever you want
#print a message after each file is saved
#----------------------------------------------
import cStringIO # *much* faster than StringIO
import urllib
from PIL import Image
y=0
for x in my_list:
    file_write = "temp" + "-" + str(y) + ".jpg"
    y = y + 1
    file = urllib.urlopen(x)
    im = cStringIO.StringIO(file.read()) # constructs a StringIO holding the image
    img = Image.open(im)
    img.save(file_write)
    print("writing file: ", file_write)
    

('writing file: ', 'temp-0.jpg')
('writing file: ', 'temp-1.jpg')
('writing file: ', 'temp-2.jpg')
('writing file: ', 'temp-3.jpg')
('writing file: ', 'temp-4.jpg')
('writing file: ', 'temp-5.jpg')
('writing file: ', 'temp-6.jpg')
('writing file: ', 'temp-7.jpg')
('writing file: ', 'temp-8.jpg')
('writing file: ', 'temp-9.jpg')
('writing file: ', 'temp-10.jpg')
('writing file: ', 'temp-11.jpg')
('writing file: ', 'temp-12.jpg')
('writing file: ', 'temp-13.jpg')
('writing file: ', 'temp-14.jpg')
('writing file: ', 'temp-15.jpg')
('writing file: ', 'temp-16.jpg')
('writing file: ', 'temp-17.jpg')
('writing file: ', 'temp-18.jpg')
('writing file: ', 'temp-19.jpg')
('writing file: ', 'temp-20.jpg')
('writing file: ', 'temp-21.jpg')
('writing file: ', 'temp-22.jpg')
('writing file: ', 'temp-23.jpg')
('writing file: ', 'temp-24.jpg')
('writing file: ', 'temp-25.jpg')
('writing file: ', 'temp-26.jpg')
('writing file: ', 'temp-27.jpg')
('writing file: ', 'temp-28.jpg')
('writing file: ', 'temp

In [62]:
#specify your Google cloud storage bucket
#use gsutil to copy the files to that bucket
bucket_storage = 'gs://' + 'open-images-urls/'
!gsutil cp '*.jpg' $bucket_storage

Copying file://temp-23.jpg [Content-Type=image/jpeg]...
Copying file://temp-0.jpg [Content-Type=image/jpeg]...                          
Copying file://temp-43.jpg [Content-Type=image/jpeg]...                         
Copying file://temp-6.jpg [Content-Type=image/jpeg]...                          
\ [4 files][  2.6 MiB/  2.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m -o ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://temp-55.jpg [Content-Type=image/jpeg]...
Copying file://temp-47.jpg [Content-Type=image/jpeg]...                         
Copying file://temp-60.jpg [Content-Type=image/jpeg]...                         
Copying file://temp-44.jpg [Content-Type=image/jpeg]...                         
Copying file://temp-57.jpg [Content-Type=image/jpeg]...              